In [1]:
import cv2
import numpy as np

def stretch_channel(channel):
    min_val = np.min(channel)
    max_val = np.max(channel)
    stretched_channel = ((channel - min_val) / (max_val - min_val) * 255).astype(np.uint8)
    return stretched_channel

def stretch_image(image):
    b, g, r = cv2.split(image)
    b_stretched = stretch_channel(b)
    g_stretched = stretch_channel(g)
    r_stretched = stretch_channel(r)
    stretched_image = cv2.merge((b_stretched, g_stretched, r_stretched))
    return stretched_image

def enhance_image(image):
    # Apply histogram equalization to enhance contrast
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab_image)
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
    l_eq = clahe.apply(l)
    enhanced_lab = cv2.merge([l_eq, a, b])
    enhanced_image = cv2.cvtColor(enhanced_lab, cv2.COLOR_LAB2BGR)
    
    # Apply color correction for underwater video
    corrected_image = gray_world(enhanced_image)
    
    return corrected_image

def gray_world(image):
    # Apply gray world assumption for color correction
    avg_b = np.mean(image[:, :, 0])
    avg_g = np.mean(image[:, :, 1])
    avg_r = np.mean(image[:, :, 2])
    
    avg_gray = (avg_b + avg_g + avg_r) / 3
    
    scale_r = avg_gray / avg_r
    scale_g = avg_gray / avg_g
    scale_b = avg_gray / avg_b
    
    balanced_image = np.zeros_like(image, dtype=np.float32)
    balanced_image[:, :, 0] = image[:, :, 0] * scale_b
    balanced_image[:, :, 1] = image[:, :, 1] * scale_g
    balanced_image[:, :, 2] = image[:, :, 2] * scale_r
    
    return np.clip(balanced_image, 0, 255).astype(np.uint8)

# Load video
video_path = "C:/Users/manda/Downloads/FOGGY Underwater video.mp4"
cap = cv2.VideoCapture(video_path)

# Check if the video opened successfully
if not cap.isOpened():
    print("Error: Couldn't open the video.")
    exit()

# Process each frame
while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # Apply filters
    stretched_frame = stretch_image(frame)
    enhanced_frame = enhance_image(stretched_frame)
    
    # Display the processed frame
    cv2.imshow('Processed Frame', enhanced_frame)
    
    # Check for 'q' key to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release video capture
cap.release()

# Destroy any OpenCV windows
cv2.destroyAllWindows()
